<a href="https://colab.research.google.com/github/Fis428/PC03/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0. 安裝必要套件（Colab 或本機皆可）
!pip install -q flask pyngrok flask_ngrok line-bot-sdk google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.0 MB/s eta 0:00:00


In [ ]:
# 2. 載入套件
from flask import Flask, request, abort
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate, CarouselColumn, URIAction, PostbackAction
)

import google.generativeai as genai


In [ ]:
# 3. 讀取環境變數
LINE_TOKEN  = os.environ['LINE_CHANNEL_ACCESS_TOKEN']
LINE_SECRET = os.environ['LINE_CHANNEL_SECRET']
GOOGLE_KEY  = os.environ['GOOGLE_API_KEY']
NGROK_TOKEN = os.environ['NGROK_AUTHTOKEN']


In [ ]:
# 4. Flask + LINE + Gemini 初始化
app = Flask(__name__)

configuration = Configuration(access_token=LINE_TOKEN)
handler       = WebhookHandler(LINE_SECRET)

genai.configure(api_key=GOOGLE_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

def ask_gemini(question: str) -> str:
    return model.generate_content(question).text


NameError: name 'LINE_TOKEN' is not defined

In [ ]:
# 5. （Colab 用）啟動 ngrok
def start_ngrok():
    ngrok.set_auth_token(NGROK_TOKEN)
    public_url = ngrok.connect(5000)
    print("ngrok URL:", public_url)
    run_with_ngrok(app)


In [ ]:
# 6. Webhook 路由
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body      = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"


In [ ]:
# 7. 處理各種訊息：確認視窗、走馬燈、或委派給 Gemini
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    action = event.message.text.strip().lower()

    # 1) 確認視窗
    if action == 'confirm':
        reply = TemplateMessage(
            alt_text="這是確認視窗",
            template=ConfirmTemplate(
                text="你喜歡看魯邦三世大冒險嗎？",
                actions=[
                    MessageAction(label="是", text="我很喜歡魯邦三世！"),
                    MessageAction(label="否", text="其實我非常喜歡，只是要假裝討厭XD")
                ]
            )
        )

    # 2) 走馬燈選單
    elif action == 'carousel':
        carousel = CarouselTemplate(columns=[
            CarouselColumn(
                thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                title='夏日大作戰',
                text='細田守導演的日本科幻動畫電影',
                actions=[
                    URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                    MessageAction(label='投票', text='我投夏日大作戰一票！')
                ]
            ),
            CarouselColumn(
                thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png',
                title='魯邦三世 卡里奧斯特羅城',
                text='宮崎駿導演的日本動畫冒險電影',
                actions=[
                    URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E6%9D%8E%E4%B8%89%E4%B8%96:%E5%8D%A1%E9%87%8C%E5%A5%A5%E6%96%AF%E7%89%B9%E7%BD%97%E5%9F%8E'),
                    MessageAction(label='投票', text='我投卡里奧斯特羅城一票！')
                ]
            )
        ])
        reply = TemplateMessage(alt_text="這是走馬燈選單", template=carousel)

    # 3) 其他文字：交給 Gemini AI 回答
    else:
        ai_text = ask_gemini(event.message.text)
        reply   = TextMessage(text=ai_text)

    # 4) 發送回覆
    with ApiClient(configuration) as api_client:
        api = MessagingApi(api_client)
        api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )


In [ ]:
# 8. 啟動整個服務
if __name__ == "__main__":
    # 若在 Colab，就啟動 ngrok；否則直接跑 Flask
    try:
        start_ngrok()
    except:
        app.run(port=5000)
